[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1CcCynZ2EJ8AXsbBi_OwRF4xCM-B5nBE-?usp=sharing)

https://huggingface.co/setu4993/LaBSE



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown --id 1elj5wSaasI2BQce7PCkk-Q4XRC1BKYeB
!gdown --id 1iAizHKe41DSYzSp20AL5-K_gXjhr4R3B
!gdown --id 1C2zR4EUDnvqqOEhIWFnxrZTZJeCqOiJk
!gdown --id 1UCMYIJZvDjZZTnaohH-A4acWZw6WA40F

Downloading...
From: https://drive.google.com/uc?id=1elj5wSaasI2BQce7PCkk-Q4XRC1BKYeB
To: /content/Mal_Train_label_5000.npy
100% 252k/252k [00:00<00:00, 36.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1iAizHKe41DSYzSp20AL5-K_gXjhr4R3B
To: /content/Mal_Train_data_5000.npy
100% 33.7M/33.7M [00:00<00:00, 81.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1C2zR4EUDnvqqOEhIWFnxrZTZJeCqOiJk
To: /content/Mal_Val_data_500.npy
100% 2.84M/2.84M [00:00<00:00, 90.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UCMYIJZvDjZZTnaohH-A4acWZw6WA40F
To: /content/Mal_val_label_500.npy
100% 28.1k/28.1k [00:00<00:00, 43.4MB/s]


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 46.5 MB/s 
     |████████████████████████████████| 3.3 MB 32.4 MB/s 
     |████████████████████████████████| 895 kB 51.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


https://huggingface.co/setu4993/LaBSE

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel
train_model = TFAutoModel.from_pretrained('setu4993/LaBSE')

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at setu4993/LaBSE.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('setu4993/LaBSE')

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.98M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import numpy as np
X_raw_train = np.load('/content/Mal_Train_data_5000.npy')
X_raw_val = np.load('/content/Mal_Val_data_500.npy')

In [ ]:
len(X_raw_train[894])

1874

In [ ]:
import math
Batch_size = 10
start = 0;Endu = Batch_size
fullsize = math.floor(len(X_raw_train)/Batch_size)

In [ ]:
for i in range(fullsize):
    if i+1 == fullsize:
        Endu = len(X_raw_train)
        encoded_input = tokenizer(list(X_raw_train)[int(start):int(Endu)], return_tensors='tf',truncation=True, padding=True,max_length=512)
        output = train_model(encoded_input)
    else:
        # print(start, Endu)
        encoded_input = tokenizer(list(X_raw_train)[int(start):int(Endu)], return_tensors='tf',truncation=True, padding=True,max_length=512)
        # print(encoded_input.shape)
        output = train_model(encoded_input)

    #For concatination only
    if i == 0:
        train_total = output.pooler_output
    else:
        #train_total = torch.cat((train_total,output.pooler_output),0) #for torch
        train_total = tf.concat([train_total,output.pooler_output],0) #for tensor
    start = Batch_size * (i+1)
    Endu = Batch_size * (i+2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
!nvidia-smi

Fri Nov 12 17:38:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    57W / 149W |   8386MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import cupy as cp

In [ ]:
cp.save('LaBSE_train.npy',train_total)

In [ ]:
# !cp example.txt /content/drive/MyDrive

In [ ]:
import math
Batch_size = 20
start = 0;Endu = Batch_size
fullsize = math.floor(len(X_raw_val)/Batch_size)

In [ ]:
for i in range(fullsize):
    if i+1 == fullsize:
        Endu = len(X_raw_val)
        encoded_input = tokenizer(list(X_raw_val)[int(start):int(Endu)], return_tensors='tf',truncation=True, padding=True,max_length=512)
        output = train_model(encoded_input)
    else:
        # print(start, Endu)
        encoded_input = tokenizer(list(X_raw_val)[int(start):int(Endu)], return_tensors='tf',truncation=True, padding=True,max_length=512)
        # print(encoded_input.shape)
        output = train_model(encoded_input)

    #For concatination only
    if i == 0:
        val_total = output.pooler_output
    else:
        #train_total = torch.cat((val_total,output.pooler_output),0) #for torch
        val_total = tf.concat([val_total,output.pooler_output],0) #for tensor
    start = Batch_size * (i+1)
    Endu = Batch_size * (i+2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [ ]:
cp.save('LaBSE_val.npy',val_total)

In [ ]:
Y_raw_train = np.load("/content/Mal_Train_label_5000.npy")
Y_raw_val = np.load("/content/Mal_val_label_500.npy")

In [ ]:
X_Embed_train = np.load('/content/LaBSE_train.npy')
X_Embed_val = np.load('/content/LaBSE_val.npy')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(Y_raw_train)
Y_val = le.fit_transform(Y_raw_val)
Y_train.shape, Y_val.shape

((4500,), (500,))

In [ ]:
!gdown --id 1za291xyvXYXh0D0hU7hlmErhw0fSiWAk

Downloading...
From: https://drive.google.com/uc?id=1za291xyvXYXh0D0hU7hlmErhw0fSiWAk
To: /content/MLAlgos.py
100% 3.05k/3.05k [00:00<00:00, 4.91MB/s]


In [ ]:
from MLAlgos import SVectorMachine, Dtree, Rforest, NB, KNN

In [ ]:
SVM_Result = SVectorMachine(X_Embed_train, Y_train, X_Embed_val, Y_val)
Dtree_Result = Dtree(X_Embed_train, Y_train, X_Embed_val, Y_val)
Rforest_Result = Rforest(X_Embed_train, Y_train, X_Embed_val, Y_val)
NB_Result = NB(X_Embed_train, Y_train, X_Embed_val, Y_val)
KNN_Result = KNN(X_Embed_train, Y_train, X_Embed_val, Y_val)

In [ ]:
import pandas as pd
BFR_Denoising= pd.DataFrame({'SVM' : SVM_Result, 
            'DTree' : Dtree_Result,
            'Rforest' : Rforest_Result,
            'NB' : NB_Result,
            'KNN' : KNN_Result})
BFR_Denoising.index = ['accuracy', 'precision', 'f1', 'recall']
BFR_Denoising = BFR_Denoising.transpose()
BFR_Denoising

,accuracy,precision,f1,recall
SVM,57.2000%,57.1427%,57.0719%,57.2000%
DTree,36.0000%,36.3444%,36.0895%,36.0000%
Rforest,48.6000%,47.8500%,47.8327%,48.6000%
NB,45.2000%,44.7133%,43.6500%,45.2000%
KNN,46.8000%,51.0027%,47.0799%,46.8000%


In [ ]:
!gdown --id 1clOMEPYmgpY1EvSyivTu6GyMZmaSatDm

Downloading...
From: https://drive.google.com/uc?id=1clOMEPYmgpY1EvSyivTu6GyMZmaSatDm
To: /content/WR_LS_Denoising.py
100% 672/672 [00:00<00:00, 1.02MB/s]


In [ ]:
from WR_LS_Denoising import WR_LS_Denoise

In [ ]:
X_train = cp.load('/content/LaBSE_train.npy')
X_val = cp.load('/content/LaBSE_val.npy')
type(X_train), type(X_val)

(cupy._core.core.ndarray, cupy._core.core.ndarray)

In [ ]:
import time

In [ ]:
start = time.ctime(time.time())
print(start)
for i in range(len(X_train)):
    Denoised =  WR_LS_Denoise(X_train[i])
    X_train[i] = Denoised #Bfr we are having this X_train[i] = X_train[i]
    if(i%1000 == 0):
        print(i, time.ctime(time.time()))

Fri Nov 12 18:04:18 2021
0 Fri Nov 12 18:04:26 2021
1000 Fri Nov 12 18:05:04 2021
2000 Fri Nov 12 18:05:42 2021
3000 Fri Nov 12 18:06:20 2021
4000 Fri Nov 12 18:06:57 2021


In [ ]:
start = time.ctime(time.time())
print(start)
for i in range(len(X_val)):
    Denoised =  WR_LS_Denoise(X_val[i])
    X_val[i] = Denoised #Bfr we are having this X_train[i] = X_train[i]
    if(i%1000 == 0):
        print(i, time.ctime(time.time()))

Fri Nov 12 18:07:18 2021
0 Fri Nov 12 18:07:18 2021


In [ ]:
X_Denoise_Embed_train = cp.ndarray.get(X_train)
X_Denoise_Embed_val = cp.ndarray.get(X_val)

In [ ]:
from MLAlgos import SVectorMachine, Dtree, Rforest, NB, KNN

SVM_Denoise_Result = SVectorMachine(X_Denoise_Embed_train, Y_train, X_Denoise_Embed_val, Y_val)
Dtree_Denoise_Result = Dtree(X_Denoise_Embed_train, Y_train, X_Denoise_Embed_val, Y_val)
Rforest_Denoise_Result = Rforest(X_Denoise_Embed_train, Y_train, X_Denoise_Embed_val, Y_val)
NB_Denoise_Result = NB(X_Denoise_Embed_train, Y_train, X_Denoise_Embed_val, Y_val)
KNN_Denoise_Result = KNN(X_Denoise_Embed_train, Y_train, X_Denoise_Embed_val, Y_val)

In [ ]:
import pandas as pd
AFTR_Denoising= pd.DataFrame({'SVM' : SVM_Denoise_Result, 
            'DTree' : Dtree_Denoise_Result,
            'Rforest' : Rforest_Denoise_Result,
            'NB' : NB_Denoise_Result,
            'KNN' : KNN_Denoise_Result})
AFTR_Denoising.index = ['accuracy', 'precision', 'f1', 'recall']
AFTR_Denoising = AFTR_Denoising.transpose()
AFTR_Denoising

,accuracy,precision,f1,recall
SVM,57.2000%,57.1427%,57.0719%,57.2000%
DTree,36.0000%,36.3444%,36.0895%,36.0000%
Rforest,50.6000%,49.9827%,49.8748%,50.6000%
NB,45.2000%,44.7133%,43.6500%,45.2000%
KNN,46.8000%,51.0027%,47.0799%,46.8000%
